# <span style="color:#004f29"><u> Optimisez la gestion des données d'une boutique avec Python </u></span> 

## <span style="color:#1a8e88"><u> 1 - Importation des librairies </u> </span>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from scipy import stats
import warnings

## <span style="color:#1a8e88"><u>2 - Importation des fichiers</u></span>

In [2]:
# Les fichier ".xlsx" entraîne un message d'avertissement "UserWarning"
# Ajout d'une ligne de code pour ignorer les avertissements de cette catégorie
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._read_only")

In [3]:
erp = pd.read_excel("erp.xlsx")
liaison = pd.read_excel("liaison.xlsx")
web = pd.read_excel("web.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'erp.xlsx'

In [ ]:
# Vérification de la version de "openpyxl"
!pip show openpyxl

In [ ]:
# Mise a jour vers la version la plus récente
!pip install --upgrade openpyxl

## <span style="color:#1a8e88"><u>3 - Visualisation du dataframe "ERP"</u></span>

### <span style="color:#2651ac">3.1 - Consultation des données</span>

In [ ]:
erp

In [ ]:
erp.info()

In [ ]:
erp.isnull().sum()

### <span style="color:#2651ac">3.2 - affichage d'une anomalie </span>

In [ ]:
#L'id 4954 indique un stock à 0 mais un statut "instock"
print(erp.loc[erp["product_id"] == 4954])

### <span style="color:#2651ac">3.3 - Correction</span>

In [ ]:
# Création d'une fonction qui va indiquer dans la colonne 'stock_status' une "rupture" si le stock est à 0 et "en stock" si la valeur est supérieur.
def etat_des_stocks(df):
    df['stock_status'] = np.where(df['stock_quantity'] == 0, 'rupture','en stock')

#### <span style="color:#0092a8">Vérification</span>

In [ ]:
etat_des_stocks(erp)
erp

In [ ]:
# Vérification de l'id 4954
print(erp.loc[erp["product_id"] == 4954])

#### <span style="color:#0092a8">J'exporte le fichier pour une nouvelle analyse</span>

In [ ]:
erp.to_excel('erp_v2.xlsx', sheet_name='EtatStock')

### <span style="color:#2651ac">3.4 - Vérification de la colonne "product_id"</span>

In [ ]:
# Création d'une fonction à double usage :
# 1 - Vérification de l'unicité d'une colonne
# 2 - Indique si il y a des doublons dans la colonne ainsi que la quantité trouvée
def verifClePrimaireEtDoublons(df, colonne):
    """Fonction qui prend en entrée un dataframe et une colonne, vérifie si cette colonne est une clé primaire valide, détecte les doublons et renvoie les doublons s'il y en a"""
    valeurs_uniques = df[colonne].nunique()
    nombre_lignes = len(df)
    doublons = df[df.duplicated(subset=colonne, keep=False)]
    if valeurs_uniques == nombre_lignes:
        print(f"La colonne '{colonne}' est une clé primaire valide")
    else:
        print(f"La colonne '{colonne}' n'est pas une clé primaire valide")

    if len(doublons)>0:
        print(f"Il y a {len(doublons)} doublons dans la colonne '{colonne}'.")
        return doublons
    else:
        print(f"Il n'y a pas de doublons dans la colonne '{colonne}'.")
        return

##### <span style="color:#0092a8">application de la fonction</span>

In [ ]:
# Utilisation de la fonction sur le dataframe "ERP" et la colonne "product_id"
verifClePrimaireEtDoublons(erp, "product_id")

## <span style="color:#1a8e88"><u>4 - Visualisation du dataframe "Liaison"</u></span>

### <span style="color:#2651ac">4.1 - Consultation des données</span>

In [ ]:
liaison

In [ ]:
liaison.info()

In [ ]:
liaison.isnull().sum()

In [ ]:
liaison.isnull()

In [ ]:
# Vérification de la colonne "product_id"
verifClePrimaireEtDoublons(liaison, "product_id")

In [ ]:
# Vérification de la colonne "id_web"
verifClePrimaireEtDoublons(liaison, "id_web")

### <span style="color:#2651ac">4.2 - Les doublons dans le dataframe "Liaison"</span>

In [ ]:
# Affichage des doublons
# Impossible d'attribuer un identifiant "id_web" aux "product_id" de manière coohérante
print(erp.loc[(erp["product_id"] == 4055) | 
              (erp["product_id"] == 4090) | (erp["product_id"] == 4092) | 
              (erp["product_id"] == 4195) | (erp["product_id"] == 4209)])

### <span style="color:#2651ac">4.3 - Création d'un dataframe avec les valeurs non null</span>

In [ ]:
liaison_erp_web = liaison[liaison.notnull()]

### <span style="color:#2651ac">4.4 - Passage de l'id_web au format "str"</span>

In [ ]:
liaison_erp_web.info()

### <span style="color:#2651ac">4.5 - Affichage du résultat</span>

In [ ]:
liaison_erp_web

## <span style="color:#1a8e88"><u>5 - Visualisation du dataframe "WEB"</u></span>

In [ ]:
web

In [ ]:
web.dtypes

In [ ]:
web.isnull().sum()

In [ ]:
# Vérification que les valeurs "null" qui ce trouvait dans le df "liaison" sont présente dans le df "web"
print(web.loc[(web["sku"] == 4055) | (web["sku"] == 4090) | 
              (web["sku"] == 4092) | (web["sku"] == 4195) | (web["sku"] == 4209)])

### <span style="color:#2651ac">5.1 - Découverte de deux valeurs sans identifiant</span>

In [ ]:
# Séléction des lignes "null" depuis la colonne "sku" avec tri depuis la colonne "post_title"
web.loc[web["sku"].isnull(),:].sort_values("post_title", ascending=False)

In [ ]:
# Triage décroissant des lignes par la colonne "SKU" puis "post_type"
web.sort_values(["sku", "post_type"], ascending=[False, False]).head(20)

### <span style="color:#2651ac">5.2 - Renommage de la colone "sku" par "id_web"</span>

In [ ]:
web.rename(columns={"sku" : "id_web"}, inplace=True)

### <span style="color:#2651ac">5.3 - Selection des lignes utilisable pour l'analyse</span>

In [ ]:
# Création d'un nouveau df avec uniquement les lignes d'origine (df : web) ayant une valeur entre "post_type" et "product"
web_pdt = web[web["post_type"] == "product"]
web_pdt

### <span style="color:#2651ac">5.4 - Affichage des doublons</span>

In [ ]:
duplicates = web_pdt[web_pdt.duplicated(subset=["id_web"], keep=False)]
display(duplicates)

### <span style="color:#2651ac">5.5 - Suppression des valeurs sans identifiant</span>

In [ ]:
web_produit = web_pdt.dropna(subset=["id_web"])

In [ ]:
web_produit

### <span style="color:#2651ac">5.6 - Suppression des colonnes ayant des valeurs unique</span>

In [ ]:
# Calcul du nombre de valeurs uniques
web_unique = web_produit.apply(pd.Series.nunique)

# Stockage des colonnes avec la condition dans une variable
drop_col = web_unique[web_unique ==1].index

# Suppression des colonnes identifiées dans l'étape précédante
web_produit = web_produit.drop(drop_col, axis=1)

#### <span style="color:#0092a8">5.6.1 - Affichage du résultat</span>

In [ ]:
web_produit.info()

In [ ]:
verifClePrimaireEtDoublons(web_produit, "id_web")

---

## <span style="color:#1a8e88"><u>6 - Les demandes de Laurent</u></span>

### <span style="color:#2651ac">6.1 - Rapprochement entre les dataframes "ERP" et "web"</span>

In [ ]:
erp.info()

### <span style="color:#2651ac">6.2 - Merge des dataframes</span>

In [ ]:
#la jointure "outer" conserve toutes les lignes des deux dataframes et précise d'ou proviennent les données.
erp_liaison = pd.merge(erp, liaison_erp_web, on="product_id", how="outer", indicator=True)
erp_liaison

In [ ]:
#Verification des valeurs null
erp_liaison.isnull().sum()

In [ ]:
# Calcul des occurences de la colonne "_merge"
merged = erp_liaison["_merge"].value_counts()
print(merged)

In [ ]:
erp_liaison[erp_liaison["_merge"]=="both"]

In [ ]:
# Création d'un df avec uniquement les valeurs non null
erp_liaison_nonull = erp_liaison[erp_liaison["id_web"].notna()]

In [ ]:
print(erp_liaison_nonull)

In [ ]:
# Calcul des occurences de la colonne "_merge"
merged = erp_liaison_nonull["_merge"].value_counts()
print(merged)

In [ ]:
# Suppression de la colonne "_merge" 
del erp_liaison_nonull["_merge"]

In [ ]:
# La jointure "outer" conserve toutes les lignes des deux dataframes et précise d'ou proviennent les données.
erp_liaison_web = pd.merge(web_produit, erp_liaison_nonull, on="id_web", how="outer", indicator=True)

In [ ]:
erp_liaison_web

In [ ]:
erp_liaison_web.isnull().sum()

In [ ]:
# Calcul des occurences de la colonne "_merge"
merged = erp_liaison_web["_merge"].value_counts()
print(merged)

In [ ]:
# Selection des lignes du dataframe ayant la condition "both" c'est a dire qu'il sont dans les deux df
erp_liaison_web = erp_liaison_web[erp_liaison_web["_merge"] == "both"]

In [ ]:
# Calcul des occurences de la colonne "_merge"
rendu_erp_web = erp_liaison_web["_merge"].value_counts()
print(rendu_erp_web)

In [ ]:
# Suppression de la colonne "_merge"
del erp_liaison_web["_merge"]

In [ ]:
erp_liaison_web.info()

## <span style="color:#1a8e88"><u>7 - Calculs </u></span>

### <span style="color:#2651ac">7.1 - Chiffre d'affaire par produits</span>

#### <span style="color:#0092a8">7.1.1 - Création et préparation du dataframe "ca_pdt"</span>

In [ ]:
# Selection des colonnes pour le df
# Renomage des colonnes
# Définition de "ID_Produits" comme index pour le nouveau dataframe
ca_pdt = erp_liaison_web[["product_id", "id_web", "post_title", "stock_quantity", "price", "onsale_web", "total_sales"]].rename(columns={"product_id":"ID_Produits", "post_title":"Titre", "stock_quantity":"Stock_dispo", "id_web":"ID_Web", "price":"Prix", "onsale_web":"Vente_Web", "total_sales":"Vente_total"})
ca_pdt.set_index("ID_Produits", inplace=True)
ca_pdt["Vente_total"].isnull().sum()

In [ ]:
# Passage des valeurs de la colonnes "Vente_total" en entier (integer)
ca_pdt["Vente_total"] = ca_pdt["Vente_total"].astype(int)

In [ ]:
ca_pdt

In [ ]:
# Calcul du Chiffre d'affaire en multipliant les valeurs de la colonne "Prix" avec les valeurs de la colonne "Vente_total" 
ca_pdt["Chiffre_affaire"] = (ca_pdt["Prix"] * ca_pdt["Vente_total"])

In [ ]:
# Classement des 10 meilleurs vente 
ca_pdt.sort_values(by=["Vente_total"], ascending=False).head(10)

### <span style="color:#2651ac">7.2 - CA total en ligne</span>

In [ ]:
# Calcul du chiffre d'affaire total
total_ca = ca_pdt["Chiffre_affaire"].sum()
print("Le chiffre d'affaire total est de :", total_ca,"€, incluant 250 € de bons-cadeaux. (10 x 25€)")

### <span style="color:#2651ac">7.3 - Détéction des valeurs abérantes</span>

### <span style="color:#2651ac">7.3.1 - Calcul des quartiles avec la fonction "quantile"</span>

In [ ]:
# Affichage du diagramme en boite (ou boite à moustache) représentant les quantiles et les outliers
plt.figure(figsize=(20,5))
plt.xticks(np.arange(5, 275, 5))
sns.boxplot(x=ca_pdt["Prix"], flierprops=dict(markerfacecolor='red', markersize=6))
plt.show()

In [ ]:
# Le quantile 0.25 représente le premier quartile qui est la valeurs inférieur à laquelle se trouve 25% des données  
Q1 = round(ca_pdt.Prix.quantile(0.25),2)
#Le quantile supérieur réprésente 25% des données les plus grandes
Q3 = round(ca_pdt.Prix.quantile(0.75),2)
print("{:.2f} €".format(Q1))
print("{:.2f} €".format(Q3))

In [ ]:
# Calcul de l'inter quartile "IQR"
IQR = Q3 - Q1
IQR

In [ ]:
#Calcul des limites inférieure et supérieure pour la détéction des valeurs abbérantes
limite_inférieure = round(Q1 - 1.5*IQR,2)
limite_supérieure = round(Q3 + 1.5*IQR,2)
limite_inférieure, limite_supérieure

In [ ]:
ca_pdt[(ca_pdt.Prix>limite_supérieure) | (ca_pdt.Prix<limite_inférieure)].sort_values(by="Prix", ascending=False)

## <span style="color:#1a8e88"><u>Conclusion</u></span>

### Cette analyse a révélé que certains tarifs pour les vins, champagnes et alcools peuvent sembler élevés, mais il est courant de trouver des produits à ces prix. Par exemple, en 2018, une bouteille de "Château Margaux" pour le millésime 2009 a pu coûter environ 1800 €, et les millésimes plus anciens peuvent coûter encore plus cher (Château Margaux 1787 à 137580 €). À l'inverse, certains vins de table peuvent être vendus à environ 1,50€ la bouteille.
### Il peut être difficile d'utiliser le terme "valeurs aberrantes" pour ce type de produits, compte tenu des différences entre un vin de table, un Grand Cru et un millésime.